<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/Self_supervised_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

In [ ]:
#!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [4]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 319 (delta 61), reused 51 (delta 28), pack-reused 230
Receiving objects: 100% (319/319), 86.33 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [5]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [7]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
num_classes = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
out_dir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
memSize = 512
alpha=1

In [ ]:
#!rm -rf "ML_DL_Project"
#!rm -rf "./GTEA61"
#!rm -rf "./GTEA61_val"

In [8]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.RegObjectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.resnetMod import *
#from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
#Prende il makeDataset dell'ultimo script importato
import argparse
import sys

In [ ]:
!rm -rf "./out_dir"

In [13]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()

In [21]:
for seqImages, seqMaps, labels in train_loader:
  print(f"Batch size: {len(seqImages)}")
  #Make dataset restituisce una sequenza di immagini e di mappe
  print(f"Batch size: {len(maps)}")
  print(f"Number of frames selected: {len(seqImages[0])}")
  print(f"Number of frames selected maps: {len(seqMaps[0])}")

  break

Batch size: 32
Batch size: 32
Number of frames selected: 7
Number of frames selected maps: 7
